In [1]:
import numpy as np
import pandas as pd
from statsmodels.tsa.vector_ar.var_model import VAR

In [2]:
data = pd.read_excel(f'data(SA){20240429}.xlsx', index_col=0)
data.index = pd.PeriodIndex(data.index, freq='M')

In [3]:
endog = data[['GDP(SA)', '전산업생산지수(SA)']]
endog = np.log(endog)
endog = endog.groupby(endog.index.asfreq('Q')).mean()
endog = endog.diff()*100

In [4]:
endog['회사채3년AA국채3년금리차'] = data['회사채3년AA국채3년금리차'].groupby(data.index.asfreq('Q')).mean()

In [5]:
endog = endog.dropna()

In [6]:
results = pd.DataFrame(
    index=pd.period_range(start='2007Q1', end='2016Q1', freq='Q'),
    columns=[
        'var_nowcasting',
        'var_nowcasting_1Q_ahead'
        ]
    )

In [7]:
for period in results.index:
    realtimeEndog = endog.loc[:period - 1].copy()
    mdl = VAR(realtimeEndog).fit()
    fcst = mdl.forecast(y=realtimeEndog.iloc[-mdl.k_ar:].to_numpy(), steps=1)
    results.loc[period, 'var_nowcasting'] = fcst[0][0]

In [8]:
for period in results.index:
    realtimeEndog = endog.loc[:period - 2].copy()
    mdl = VAR(realtimeEndog).fit()
    fcst = mdl.forecast(y=realtimeEndog.iloc[-mdl.k_ar:].to_numpy(), steps=2)
    results.loc[period, 'var_nowcasting_1Q_ahead'] = fcst[1][0]

In [9]:
results

,var_nowcasting,var_nowcasting_1Q_ahead
2007Q1,2.154399,0.99412
2007Q2,0.73021,1.034079
2007Q3,1.19287,1.161066
2007Q4,1.566849,1.228298
2008Q1,1.654123,1.172038
2008Q2,1.522459,1.332697
2008Q3,0.916612,1.459869
2008Q4,1.121786,1.33451
2009Q1,-2.692955,1.343839
2009Q2,0.750234,-3.677332


In [10]:
results.to_excel('var_results.xlsx')